In [64]:
import importlib
import codecs
import os
import pandas
import numpy 
import datetime as dt
import scipy.stats
import pandasql.sqldf
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.max_rows = 200
pandas.options.display.max_columns = 50
pandas.options.display.width = 300
pandas.set_option('display.float_format', lambda x: '%.8f' % x)

import trm3.impute
import trm3.utility

input_file = r"D:\projects\Python36-Package-Development\trm3pylib\data-csv\zillow_train.csv"
with codecs.open(input_file, 'r', encoding='utf-8') as f:
   df= pandas.read_csv(f, sep=",", header=0, low_memory=False, parse_dates=True)

df["datecol"] = pandas.to_datetime(df.transactiondate)
df["interval"] = df["datecol"] - dt.datetime.now().date()
df["fips"].fillna(value=df["fips"].mode().tolist()[0], inplace=True)


In [65]:
importlib.reload(trm3.impute)
importlib.reload(trm3.utility)

<module 'trm3.impute' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\impute.py'>

<module 'trm3.utility' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\utility.py'>

In [66]:
sdf = trm3.utility.df_contents(df, display_html_output=True)
#                   excel_filename=r"D:\projects\Python36-Package-Development\temp\test01.xlsx",
#                   excel_sheet_name="DF Summary")

,column_dtype,n_missing,n_non_missing,n_boolean,n_float,n_integer,n_string,n_datetime,n_timedelta,n_unique_values,mode,mean,stdev,min,p1,p5,p10,p25,median,p75,p90,p95,p99,max
Column Name,,,,,,,,,,,,,,,,,,,,,,,,
airconditioningtypeid,float64,61934,28748,0,28748,0,0,0,0,7,[1.0],1.81522193,2.97210778,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,1.00000000,13.00000000,13.00000000,13.00000000
architecturalstyletypeid,float64,90422,260,0,260,0,0,0,0,7,[7.0],7.23076923,2.72139659,2.00000000,2.00000000,3.00000000,7.00000000,7.00000000,7.00000000,7.00000000,7.00000000,8.00000000,21.00000000,21.00000000
assessmentyear,float64,532,90150,0,90150,0,0,0,0,2,[2015.0],2015.00000000,0.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000,2015.00000000
basementsqft,float64,90639,43,0,43,0,0,0,0,40,[1528.0],713.58139535,437.43419818,100.00000000,126.04000000,169.60000000,196.40000000,407.50000000,616.00000000,872.00000000,1492.40000000,1528.00000000,1553.32000000,1555.00000000
bathroomcnt,float64,532,90150,0,90150,0,0,0,0,24,[2.0],2.27954520,1.00413283,0.00000000,0.00000000,1.00000000,1.00000000,2.00000000,2.00000000,3.00000000,3.00000000,4.00000000,5.50000000,20.00000000
bedroomcnt,float64,532,90150,0,90150,0,0,0,0,18,[3.0],3.03193566,1.15611372,0.00000000,0.00000000,1.00000000,2.00000000,2.00000000,3.00000000,4.00000000,4.00000000,5.00000000,6.00000000,16.00000000
buildingclasstypeid,float64,90666,16,0,16,0,0,0,0,2,[4.0],4.00000000,0.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000,4.00000000
buildingqualitytypeid,float64,33398,57284,0,57284,0,0,0,0,9,[7.0],5.56539348,1.90044284,1.00000000,1.00000000,4.00000000,4.00000000,4.00000000,7.00000000,7.00000000,7.00000000,7.00000000,10.00000000,12.00000000
calculatedbathnbr,float64,1708,88974,0,88974,0,0,0,0,23,[2.0],2.30917459,0.97613689,1.00000000,1.00000000,1.00000000,1.00000000,2.00000000,2.00000000,3.00000000,3.00000000,4.00000000,5.50000000,20.00000000


In [5]:
df.columns.tolist()

['parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',
 'firep

In [76]:
importlib.reload(trm3.impute)    
my_imputer = trm3.impute.Imputer(train_df = df,
                                 test_df = None,
                                 columns = [x for x in df.columns.tolist() if x[0:3] != "dv_" and x !="fips"],
#                                  columns = ["datecol"],
                                 groupby = ["fips"],
                                 label = "dv_avg_error",
                                 method = 4,
                                 mode_fallback_distinct_values = 3,
                                 mean_label_min_obs = .005,
                                 mean_label_bins = 10,
                                 nominal_new_category_min_obs = .01,
                                 missing_indicators = False,
                                 missing_indicators_min_obs = .01,
                                 model_x_columns = None                          
                                )

my_imputer_rules = my_imputer.fit()
# my_imputer_rules.impute(impute_df=df, inplace=False)

<module 'trm3.impute' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\impute.py'>

DEBUG:  1 - parcelid
[parcelid] -> 1.174607753753662 seconds
DEBUG:  2 - airconditioningtypeid
[airconditioningtypeid] -> 1.1736316680908203 seconds
DEBUG:  3 - architecturalstyletypeid
[architecturalstyletypeid] -> 1.1908788681030273 seconds
DEBUG:  4 - basementsqft
[basementsqft] -> 1.19313383102417 seconds
DEBUG:  5 - bathroomcnt
[bathroomcnt] -> 1.1976420879364014 seconds
DEBUG:  6 - bedroomcnt
[bedroomcnt] -> 1.1506879329681396 seconds
DEBUG:  7 - buildingclasstypeid
[buildingclasstypeid] -> 1.187178373336792 seconds
DEBUG:  8 - buildingqualitytypeid
[buildingqualitytypeid] -> 1.153611660003662 seconds
DEBUG:  9 - calculatedbathnbr
[calculatedbathnbr] -> 1.1996674537658691 seconds
DEBUG:  10 - decktypeid
[decktypeid] -> 1.1897399425506592 seconds
DEBUG:  11 - finishedfloor1squarefeet
[finishedfloor1squarefeet] -> 1.1921346187591553 seconds
DEBUG:  12 - calculatedfinishedsquarefeet
[calculatedfinishedsquarefeet] -> 1.1531083583831787 seconds
DEBUG:  13 - finishedsquarefeet12
[finis

D:\applications\Anaconda3\lib\site-packages\pandas\core\groupby.py:3154: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


[heatingorsystemtypeid] -> 1.1711294651031494 seconds
DEBUG:  24 - latitude
[latitude] -> 1.1538360118865967 seconds
DEBUG:  25 - longitude
[longitude] -> 1.1390972137451172 seconds
DEBUG:  26 - lotsizesquarefeet
[lotsizesquarefeet] -> 1.2244577407836914 seconds
DEBUG:  27 - poolcnt
[poolcnt] -> 1.186126708984375 seconds
DEBUG:  28 - poolsizesum
[poolsizesum] -> 1.1885287761688232 seconds
DEBUG:  29 - pooltypeid10
[pooltypeid10] -> 1.1971354484558105 seconds
DEBUG:  30 - pooltypeid2
[pooltypeid2] -> 1.246068000793457 seconds
DEBUG:  31 - pooltypeid7
[pooltypeid7] -> 1.187635898590088 seconds
DEBUG:  32 - propertycountylandusecode
[propertycountylandusecode] -> 0.13312530517578125 seconds
DEBUG:  33 - propertylandusetypeid
[propertylandusetypeid] -> 1.1386682987213135 seconds
DEBUG:  34 - propertyzoningdesc
[propertyzoningdesc] -> 0.18317174911499023 seconds
DEBUG:  35 - rawcensustractandblock
[rawcensustractandblock] -> 1.1831233501434326 seconds
DEBUG:  36 - regionidcity
[regionidcity

D:\applications\Anaconda3\lib\site-packages\pandas\core\groupby.py:3154: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


[structuretaxvaluedollarcnt] -> 1.1886417865753174 seconds
DEBUG:  51 - taxvaluedollarcnt
[taxvaluedollarcnt] -> 1.1791331768035889 seconds
DEBUG:  52 - assessmentyear
[assessmentyear] -> 1.1611201763153076 seconds
DEBUG:  53 - landtaxvaluedollarcnt
[landtaxvaluedollarcnt] -> 1.258683443069458 seconds
DEBUG:  54 - taxamount
[taxamount] -> 1.225184679031372 seconds
DEBUG:  55 - taxdelinquencyflag
[taxdelinquencyflag] -> 0.10610127449035645 seconds
DEBUG:  56 - taxdelinquencyyear
[taxdelinquencyyear] -> 1.223151445388794 seconds
DEBUG:  57 - censustractandblock
[censustractandblock] -> 1.1636052131652832 seconds
DEBUG:  58 - logerror
[logerror] -> 1.2136530876159668 seconds
DEBUG:  59 - transactiondate
[transactiondate] -> 0.1121056079864502 seconds
DEBUG:  60 - last_error
[last_error] -> 1.1305673122406006 seconds
DEBUG:  61 - datecol
DEBUG:  62 - interval


In [67]:
df.groupby(["fips"])["parcelid"].\
transform(lambda x: pandas.qcut(x, 10, labels=False, duplicates="drop"))

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
32       0
33       0
34       0
35       0
36       0
37       0
38       0
39       0
40       0
41       0
42       0
43       0
44       0
45       0
46       0
47       0
48       0
49       0
50       0
51       0
52       0
53       0
54       0
55       0
56       0
57       0
58       0
59       0
60       0
61       0
62       0
63       0
64       0
65       0
66       0
67       0
68       0
69       0
70       0
71       0
72       0
73       0
74       0
75       0
76       0
77       0
78       0
79       0
80       0
81       0
82       0
83       0
84       0
85       0
86       0
87       0
88       0
89       0
90       0

In [61]:
len(df)

90682

In [ ]:
df.columns[0]

In [ ]:
df["assessmentyear"].count()

In [ ]:
col = "transactiondate"
t = df[col][pandas.notnull(df[col])].apply(type)
w= t.groupby(t).count()
print(str(w.index[0]) + " - " + str(w[0]))
w

In [ ]:
if w.index[0] == pandas.Timestamp:
    print("True")

In [ ]:
df.groupby(["hashottuborspa"])["hashottuborspa"].apply(type).count()

In [ ]:
list(range(0,1))

In [ ]:
df.dtypes

In [ ]:
type(df[col].dtype)

In [ ]:
numpy.dtype('O')